# Cleanup Resources

Each notebook has a resource cleanup section that is designed to remove just the resources it created.

This notebook is designed to remove all resources by the type created.  By being a general cleanup utility, you can skip sections that you want to remain in use.  This notebook also has a section to delete the GCS bucket created in the `00 - Initial Setup` notebook.

**Resources**
- Vertex AI: https://googleapis.dev/python/aiplatform/latest/index.html
- BigQuery: https://googleapis.dev/python/bigquery/latest/index.html
- GCS: https://googleapis.dev/python/storage/latest/client.html

---
## Setup

Parameters:

In [1]:
PROJECT_ID = 'statmike-mlops'
REGION = 'us-central1'
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

AI Platform Client Setup:

In [32]:
from google.cloud import aiplatform
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}
aiplatform.init(project=PROJECT_ID, location=REGION)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

---
## BigQuery
- Delete the dataset `digits`

Make a client connection to BigQuery

In [3]:
from google.cloud import bigquery
clients['bq'] = bigquery.Client(project=PROJECT_ID)

list of datasets in the project:

In [11]:
datasets = list(clients['bq'].list_datasets())

delete each dataset:

In [10]:
for ds in datasets:
    print(ds.dataset_id)
    clients['bq'].delete_dataset(ds.dataset_id,delete_contents=True,not_found_ok=True)

digits
digits_training
prediction_bq_digits_code_20210708172309_2021_07_09T06_22_11_114Z
prediction_bq_digits_console_202178154054_2021_07_08T13_05_47_714Z


---
## Vertex AI > Datasets
- Delete datasets

In [28]:
datasets = aiplatform.TabularDataset.list()

In [29]:
for ds in datasets:
    print(ds.resource_name)
    aiplatform.TabularDataset(dataset_name=ds.resource_name).delete()

projects/691911073727/locations/us-central1/datasets/7427887140882612224
INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/691911073727/locations/us-central1/datasets/7427887140882612224
INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/691911073727/locations/us-central1/operations/806467864595791872
INFO:google.cloud.aiplatform.base:TabularDataset deleted. . Resource name: projects/691911073727/locations/us-central1/datasets/7427887140882612224
projects/691911073727/locations/us-central1/datasets/3579561259294523392
INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/691911073727/locations/us-central1/datasets/3579561259294523392
INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/691911073727/locations/us-central1/operations/6884075561732276224
INFO:google.cloud.aiplatform.base:TabularDataset deleted. . Resource name: projects/691911073727/locations/us-central1/datasets/3579561259294523392
pro

---
## Vertex AI > Feature Store
- delete feature store including entity types and features

In [34]:
from google.cloud import aiplatform_v1beta1
clients['fs'] = aiplatform_v1beta1.FeaturestoreServiceClient(client_options=client_options)

In [36]:
fs = clients['fs'].list_featurestores(parent=PARENT)

In [41]:
for f in fs:
    print(f.name)
    clients['fs'].delete_featurestore(request = aiplatform_v1beta1.types.DeleteFeaturestoreRequest(name=f.name,force=True))

projects/691911073727/locations/us-central1/featurestores/digits_featurestore


---
## Vertex AI > Endpoints
- Delete endpoints forcing deployed models to undeploy

In [15]:
endpoints = aiplatform.Endpoint.list()

In [20]:
for ep in endpoints:
    print(ep.resource_name)
    aiplatform.Endpoint(endpoint_name=ep.resource_name).delete(force=True)

projects/691911073727/locations/us-central1/endpoints/6358150287986786304
INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/691911073727/locations/us-central1/endpoints/6358150287986786304
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/6358150287986786304/operations/436328270221279232
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/691911073727/locations/us-central1/endpoints/6358150287986786304
INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/691911073727/locations/us-central1/endpoints/6358150287986786304
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/691911073727/locations/us-central1/operations/2380475934361780224
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/691911073727/locations/us-central1/endpoints/6358150287986786304
projects/691911073727/locations/us-central1/endpo

---
## Vertex AI > Models
- Delete uploaded models

In [21]:
models = aiplatform.Model.list()

In [25]:
for model in models:
    print(model.resource_name)
    aiplatform.Model(model_name=model.resource_name).delete()

projects/691911073727/locations/us-central1/models/6423689977095258112
INFO:google.cloud.aiplatform.base:Deleting Model : projects/691911073727/locations/us-central1/models/6423689977095258112
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/691911073727/locations/us-central1/operations/3382526851451715584
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/691911073727/locations/us-central1/models/6423689977095258112
projects/691911073727/locations/us-central1/models/1522647682609315840
INFO:google.cloud.aiplatform.base:Deleting Model : projects/691911073727/locations/us-central1/models/1522647682609315840
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/691911073727/locations/us-central1/operations/6913348959310184448
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/691911073727/locations/us-central1/models/1522647682609315840
projects/691911073727/locations/us-central1/models/85550185007483453

---
## Vertex AI > Batch Predictions
- delete all batch prediction jobs

In [45]:
bps = aiplatform.BatchPredictionJob.list()

In [49]:
for bp in bps:
    print(bp.resource_name)
    aiplatform.BatchPredictionJob(batch_prediction_job_name=bp.resource_name).delete()

projects/691911073727/locations/us-central1/batchPredictionJobs/3040831348009861120
INFO:google.cloud.aiplatform.base:Deleting BatchPredictionJob : projects/691911073727/locations/us-central1/batchPredictionJobs/3040831348009861120
INFO:google.cloud.aiplatform.base:Delete BatchPredictionJob  backing LRO: projects/691911073727/locations/us-central1/operations/2290403941814370304
INFO:google.cloud.aiplatform.base:BatchPredictionJob deleted. . Resource name: projects/691911073727/locations/us-central1/batchPredictionJobs/3040831348009861120
projects/691911073727/locations/us-central1/batchPredictionJobs/5978867154915688448
INFO:google.cloud.aiplatform.base:Deleting BatchPredictionJob : projects/691911073727/locations/us-central1/batchPredictionJobs/5978867154915688448
INFO:google.cloud.aiplatform.base:Delete BatchPredictionJob  backing LRO: projects/691911073727/locations/us-central1/operations/624072079687286784
INFO:google.cloud.aiplatform.base:BatchPredictionJob deleted. . Resource nam

---
## Vertex AI > Training
- delete various jobs types
- some create with aiplatform.* and other created by aiplatform.gapic.*

In [68]:
jobsA = aiplatform.CustomTrainingJob.list()
jobsA

 resource name: projects/691911073727/locations/us-central1/trainingPipelines/2717979549722738688]

In [73]:
jobsB = aiplatform.CustomPythonPackageTrainingJob.list()
jobsB

 resource name: projects/691911073727/locations/us-central1/trainingPipelines/2717979549722738688]

In [70]:
jobsC = aiplatform.AutoMLTabularTrainingJob.list()
jobsC

 resource name: projects/691911073727/locations/us-central1/trainingPipelines/3915092625673158656,
 resource name: projects/691911073727/locations/us-central1/trainingPipelines/8425729127461421056]

## Vertex AI > Training > Custom Job
- Delete custom training jobs

In [71]:
clients['job'] = aiplatform.gapic.JobServiceClient(client_options=client_options)

In [72]:
for x in clients['job'].list_custom_jobs(parent=PARENT):
    print(x.display_name)
    clients['job'].delete_custom_job(name=x.name)

06_AIP_DIGITS_20210706150049-custom-job
05_AIP_DIGITS_20210706142708


## Vertex AI > Training > Training Pipeline (Python Package)
- Delete training pipelines

In [74]:
clients['pipeline'] = aiplatform.gapic.PipelineServiceClient(client_options=client_options)

In [75]:
for x in clients['pipeline'].list_training_pipelines(parent=PARENT):
    print(x.display_name)
    clients['pipeline'].delete_training_pipeline(name=x.name)

bq_digits_code_20210708172309
bq_digits_console_202178154054
06_AIP_DIGITS_20210706150049


---
## Storage Bucket Removal

- delete contents and storage bucket `statmike-mlops` (named after project)

In [42]:
from google.cloud import storage
clients['gcs'] = storage.Client()

In [43]:
bucket = clients['gcs'].get_bucket(PROJECT_ID)

In [44]:
bucket.delete(force=True)